![Explorer](./img/Explorer.png)
# Text Data Exploration
## Introduction
For this article, I wanted to cover a few key concepts unique to handling text data.

I say unique to NLP because unlike the pre-processing of visual or audio data, the pre-processing of text data requires a few more steps. Even if the data is valid (by valid, I mean text, characters), it might not be in the language you understand (i.e. English), it might not be in Latin alphabet or, it might not be in the right encoding format (in `UTF-8` for example). If you are dealing with web scrapped data, most of the time you'll encounter `<html>` or other formatting tags that will need to be dealt with as they are not part of the content *per se*. Not to say this information is not relevant, it can reveal useful patterns, but if treated like any other word, it will most likely generate noise. So it is important to pick those up early enough.

Unfortunately, one of the solutions for this kind of data processing is Regular Expressions or `regex`... I will try and avoid this as much as possible as it hurts my brain but, I have to admit there are a few, easy to remember, patterns that will help greatly during the cleaning process and also during the tokenization process.

## Definitions
Before we start, let's talk about the lingo:
- Corpus - It is a collection of (similar) documents.
- Document - It is a collection of sentences that have the same context. It can be a review, a paragraph, a log file, etc.
- NLP - Natural Language Processing. It is an area of AI which deals with interpreting human language. In our context, it refers to the analysis of text data by a computer.
- NLTK - Natural Language Tool Kit. It is a very powerful library for NLP.

## The dataset
To support this article, I am going to use a dataset from Kaggle: [60k Stack Overflow Questions with Quality Rating](https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate). Ultimately I'll probably try to classify them but, as I found this dataset today, it is exploration time! ![Put your hat on!](./img/Indi.jpg)

The dataset covers 60,000 questions asked on [StackOveflow](https://stackoverflow.com/) between 2016 and 2020. To link this back to the lingo, this is our corpus, and a specific question is going to be called a document.

Looking at the [`Tasks`](https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate/tasks?taskId=1889) tab, we are challenged to classify SO questions based on the text quality, with a hint about the last column being added to ease a supervised classification.
### Structure
It comes as a file named `data.csv` containing 60,000 rows and 5 columns (+ 1 `Id` column which will be our index). When displayed, it looks like this:

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import string


df = pd.read_csv('data/raw/data.csv', index_col='Id')
df

,Title,Body,Tags,CreationDate,Y
Id,,,,,
34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
...,...,...,...,...,...
60467932,C++ The correct way to multiply an integer and...,<p>I try to multiply an integer by a double bu...,<c++>,2020-02-29 17:46:41,LQ_CLOSE
60468018,How can I make a c# application outside of vis...,<p>I'm very new to programming and I'm teachin...,<c#><visual-studio>,2020-02-29 17:55:56,LQ_CLOSE
60468378,WHY DJANGO IS SHOWING ME THIS ERROR WHEN I TRY...,*URLS.PY*\r\n //URLS.PY FILE\r\n fro...,<django><django-views><django-templates>,2020-02-29 18:35:39,LQ_EDIT


As with most data, displaying raw data is quite a revealing step. I think it is even more important with text data as not only do we need to understand the metadata:
- is there any `Nan` values? (`df.describe()` has already told me there is no gap in the data)
- any out of place values? (like a 0 meaning no records for example)
- etc.

But we also need to analyse and understand the actual value of the field. As we can see in the table above, the following columns are available:
- `Title` which seems to only contain text data. It appears to be the subject of the post, in English.
- `Body` which seems to contain text data but in various "format/language". We can se the `<p>` html flag, the `\r\n` tag which probably the "code" cell formatting.
- `Tags` which seems to contain the various tags one can flag a question with.
-  `CreationDate` which seems to be the posting date. The Kaggle description mentions editing of the post as a metric for post quality. This might be worth keeping in mind when dealing with the data.
- `Y` which seems to be the manually added field, the one with the labels (for supervised classification).

### Text cleaning
The dataset sems to be quite neat, the only field really needing work is the `Body` one. The date will be parsed as date later on, the tags can easily be extracted (Who said wordclouds? :angel:) and the `Y` probably should be OneHotEncoded for the classification step.

In the `Body` column, we can see the `<html>` tag are enclosed in the formatting `\r\n` (or when we are lucky the code is placed within `<code>` tags) with the exceptions of the `<p>` tag which delimits the Body field in most cases. We also notice some characters are escaped: `\'`.

Because I am more interested in the human factor - but also because I am not qualified to review 60k snippets of code -, I am going replace the code cells by a tag that won't affect the structure of the text, without destructing the surrounding information. Maybe subitting some code is a sign of quality on SO, who knows?

There are a few ways to do this. One of them is to build a cleaning function and use the `.apply(lambda x: cleaning_function(x))` method on the column to clean. It is indicated for more complex cleaning.

Another way to clean text is to use the `.replace()` method on pandas' series which is straight forward and allows regex.
![Replacements](./img/Replacements.png)

In [2]:
# This functions addresses the \r\n blocs and converts them to CODE
def regex(text):
    pattern = "r'\\t(.*?)\\t'"
    text = re.sub(pattern, " ", text)
    pattern = "\r\n(.*?)\r\n"
    return re.sub(pattern, " CODE ", text)

# The advantage of using BeautifulSoup is that all the <html> tags are parsed and disappear.
def code_block(field):
    soup = BeautifulSoup(field)
    for f in soup.findAll('code'):
        f.replaceWith('CODE')
    return (soup.get_text()).replace('\n',' ')

# df['Body'].replace('<br/>\r\n','.', inplace=True)
# df['Body'].replace('**', ' ', inplace=True)
# df['Body'].replace("\'", "'", inplace=True)
# df['Body'].replace('<p>', '.' inplace=True)
# df['Body'].replace('\n', ' ', inplace=True)
# df['Body'].replace('\ +', ' ', regex=True, inplace=True)

In [3]:
df['Body'] = df['Body'].apply(lambda x: regex(x))

In [4]:
df['Body'] = df['Body'].apply(lambda x: code_block(x))

As we can see, our `Body` field has been sanitized and, although there are repetitions in the `CODE` tag, we have managed to significantly reduce the noise generated by the various code snippets. A few broken tags/white spaces remain but they will be removed when we remove stopwords and words smaller than 3 letters. We will need to be cautious with remove the small words as programming languages such as "R" might be important.

Finally, another thing worth noting, the text is significantly harder to read by a human being (mainly because our regex replacements did an ok job but it wasn't perfect).

In [5]:
print('-----FROM-----')
print(r'I am having 4 different tables like \r\nselect * from System \r\n \r\nselect * from Set \r\nselect * from Item \r\nselect * from Versions \r\n\r\nNow for each system Id there will be **n no.of Sets**, and foe **each set** there qill be **n no. of Items** and for **each item** there will be **n no.of Versions**.\r\n\r\n**each system has  <br/>\r\nn no of set <br/>\r\neach Set has <br/>\r\nn no of Items <br/>\r\neach Item has <br/>\r\nn no of Versions**\r\n\r\n\r\nSo, Now when i give **SystemId** then i have to retrieve all the records from \r\n\r\n**Set and Items of each set and Versions of each Items** in single storedprocedure.')
print('------TO------')
df['Body'][34552974]

-----FROM-----
I am having 4 different tables like \r\nselect * from System \r\n \r\nselect * from Set \r\nselect * from Item \r\nselect * from Versions \r\n\r\nNow for each system Id there will be **n no.of Sets**, and foe **each set** there qill be **n no. of Items** and for **each item** there will be **n no.of Versions**.\r\n\r\n**each system has  <br/>\r\nn no of set <br/>\r\neach Set has <br/>\r\nn no of Items <br/>\r\neach Item has <br/>\r\nn no of Versions**\r\n\r\n\r\nSo, Now when i give **SystemId** then i have to retrieve all the records from \r\n\r\n**Set and Items of each set and Versions of each Items** in single storedprocedure.
------TO------


'I am having 4 different tables like  CODE   CODE select * from Item  CODE  CODE  CODE n no of set  CODE n no of Items  CODE n no of Versions** CODE  CODE \r **Set and Items of each set and Versions of each Items** in single storedprocedure.'

In [6]:
df['Body'].replace(" +", " ", regex= True, inplace= True)
df['Body'].replace(r'c\+\+|C\+\+|cpp','Cpp', regex= True, inplace= True)
df['Body'].replace(r'c\#|C\#','Csharp', regex= True, inplace= True)
df['Body'].replace('R', 'Rrrrr', inplace= True)

# Removing stopwords

In [7]:
stop_words = list(string.punctuation)
stop_words += stopwords.words('english')

In [8]:
# from nltk import word_tokenize
# def toklowstop(text):
#     tokens = word_tokenize(text)
#     # converts to lower case
#     tokens = [tok.lower() for tok in tokens if len(tok) >= 3]
#     # removes the stopwords
#     words = [word for word in tokens if word not in stop_words]
#     return words
# df['Body']=df['Body'].apply(lambda x: toklowstop(x))

In [9]:
df.Body[34552974]

'I am having 4 different tables like CODE CODE select * from Item CODE CODE CODE n no of set CODE n no of Items CODE n no of Versions** CODE CODE \r **Set and Items of each set and Versions of each Items** in single storedprocedure.'

In [10]:
df.to_csv('data/processed/data.csv')